# Starting by building the game and the user interface

In [ ]:
import numpy as np

In [ ]:
def showBoard():
    '''Shows the board'''
    for val in board:
        print(val)

def checkFree(x,y):
    '''Takes an x,y postion and checks if that point on the board is free'''
    if board[x,y] ==0:
        return True
    else:
        return False
def checkWin(toggle):
    '''checks wins for each player for diagonals, rows, columns'''
    if toggle:
        high = 6
    else:
        high = 15
    if board.diagonal().sum() == high:
        return True
    if np.flip(board,1).diagonal().sum() == high:
        return True
    for val in range(0,3):
        if board[val,:].sum()==high:
            return True
    for val in range(0,3):
        if board[:,val].sum()==high:
            return True
    return False

def getAvailablePositions():
    pos = []
    for i in range(3):
        for j in range(3):
            if checkFree(i,j):
                pos.append([i,j])
    return pos

def placePiece(x,y,nought_or_cross):
    '''Takes a x, y position and a X or O with X=1, and O=2'''
    board[x,y] = nought_or_cross

In [ ]:
#run function
toggle = False
board = np.zeros((3,3))
expert = [[0,1],[0,2],[1,2],[2,2],[2,1],[0,0],[1,0],[1,1],[2,0]]
q_values = init()
count = 0
while True:
    #showBoard()
    toggle = not toggle
    
    if toggle:
        x = input('Please enter X')
        y = input('Please enter Y')
        while not checkFree(int(x),int(y)):
            print('Please enter a free location!')
            x = input('Please enter X')
            y = input('Please enter Y')
    else:
        x = expert[count][0]
        y = expert[count][1]
        count = count+1
        while not checkFree(x,y):
            x = expert[count][0]
            y = expert[count][1]
            count = count+1
        print('Expert turn')

    if toggle == True:
        n_or_c = 5
    else:
        n_or_c = 2
    placePiece(int(x),int(y),n_or_c)
    if checkWin(toggle):
        print('2 has won!')
        break
    if checkWin(not toggle):
        print('5 has won!')
        break

showBoard()

In [ ]:
def reward(result):
    '''Reward a game won or lost'''
    if result == 'lost':
        return -10
    else:
        return 10

#### Initial thoughts: Have an set of actions to choose from. i.e the available board positions and either randomly pick one or choose the one that provided the best reward in the past. This means we need to define rewards for each individual placements. But we are not taking into account the temporal element as well as the rest of the board. Is this important? I would think so. Or is there a list of optimal positions in descending order so if one is chosen then choose the next best in the list.

#### Other option is to have a reward for each game: Then each set of moves made by the agent is rated. The reward per each move is then summed up and you get the distribution for each move. Then the agent can select the best moves based on the rewards.  

#### This also means I have to play thousands of games to teach the agent. Otherwise I have to create a good expert. We could have an expert that plays a specific game

In [ ]:
def init():
    '''Initilizes the q-values'''
    q_val = {}
    expert = [[0,1],[0,2],[1,2],[2,2],[2,1],[0,0],[1,0],[1,1],[2,0]]
    for val in expert:
        q_val[str(val)]=0
    return q_val

def predict(avail_moves,epsilon,q_val):
    '''Takes the available moves, epsilon and q_values
        Chooses a random move from the available moves if the random number is higher than epsilon. 
        Chooses the available move which has the highest q_val 
        Returns the move chosen
    '''
    ran_num = np.random.rand()
    if ran_num > epsilon:
        return np.random.choice(avail_moves)
    else:
        max = 0
        for val in avail_moves:
            if q_val[str(val)]>max:
                max = q_val[str(val)]
                best = val
        return best

def updateQValues(movesPerformed,q_val,reward,gameNum):
    for move in movesPerformed:
        q_val[str(move)] = (q_val[str(move)]+reward)/gameNum
    return q_val
    